In [19]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

In [20]:
#Conjunto de dados Iris
dados=load_iris().data
classe=load_iris().target

In [21]:
#Separação em treino e teste estratificado pela classe e sempre a mesma partição.
x_treino,x_teste,y_treino,y_teste=train_test_split(dados,classe,test_size=0.33
                                            ,stratify=classe,random_state=7)

In [22]:
#Sempre a mesma amostra das observações que não terão classe para todos os k's.
np.random.seed(7)
#Esse comando escolhe aleatoriamente metade das posições do conjunto de treinamento
#que não terão classe.
posicoes_sem_classe=np.random.choice(len(y_treino),replace=False,
                                     size=int(np.floor(len(y_treino)/2)))

In [23]:
#Atribuição da classe com -1, essas observações serão as que não terão classe.
y_treino[posicoes_sem_classe]=-1

In [24]:
class classificador_semi_sup():
    def __init__(self,k,classificador):
        self.k=k
        self.classificador=classificador               
        if self.k>0 and type(self.k)==int and self.k<=len(posicoes_sem_classe):
            pass
        else:
            print("Voce nao digitou as quantidades pedidas adequadamente, tente denovo!")
                  
    def fit(self,x_treino,y_treino):  
        self.x_treino = x_treino
        self.y_treino = y_treino
        #Comando que identifica as observações sem classe.
        posicoes_sem_classe=np.where(self.y_treino==-1)[0] 
        #No caso em que o o número de observações não é múltiplo de k, esse 
        #passo resolve esse problema, na última iteração ele pega todas as 
        #observações sem classe que restam.              
        ultimo_indice=int(np.ceil(len(posicoes_sem_classe)/self.k))   
        for i in range(ultimo_indice):
            self.modelo=self.classificador()
            
            #Última iteração.
            if i==ultimo_indice-1:
                #O treino provisório é o conjunto de observações em que as posições
                #são a diferença entre o conjunto com todas as observações de treinamento
                #e do conjunto que não possuem classe, logo irá resultar nas observações
                #que contenham a classe.
                x_treino_provisorio=(self.x_treino[np.setdiff1d(range
                       (len(self.y_treino)),posicoes_sem_classe),:])
                y_treino_provisorio=(self.y_treino[np.setdiff1d(range
                       (len(self.y_treino)),posicoes_sem_classe)])
                self.modelo.fit(x_treino_provisorio,y_treino_provisorio)
                #Depois do algoritmo estar treinado ele prediz as classes das
                #k observações sem classe ou das observações que restam.
                self.y_treino[posicoes_sem_classe]=(self.modelo.predict
                        (self.x_treino[posicoes_sem_classe,:]))
            #Todas as iterações menos a última.   
            else:
                #O treino provisório é o conjunto de observações em que as posições
                #são a diferença entre o conjunto com todas as observações de treinamento
                #e do conjunto que não possuem classe, logo irá resultar nas observações
                #que contenham a classe.
                x_treino_provisorio=(self.x_treino[np.setdiff1d(range
                             (len(self.y_treino)),posicoes_sem_classe),:])
                y_treino_provisorio=(self.y_treino[np.setdiff1d(range
                             (len(self.y_treino)),posicoes_sem_classe)])
                self.modelo.fit(x_treino_provisorio,y_treino_provisorio)
                #k posicoes aleatorias que serao atribuidas as observacoes sem classe.
                posicoes_classificadas=(np.random.choice(posicoes_sem_classe
                ,replace=False,size=self.k))
                #Depois do algoritmo estar treinado ele prediz as classes das
                #k observações sem classe ou das observações que restam.
                self.y_treino[posicoes_classificadas]=(self.modelo.predict
                        (self.x_treino[posicoes_classificadas,:]))
                #Aqui o array das posições sem classe é atualizado para que 
                #somente contenham posições de observações sem classe em cada iteração.
                posicoes_sem_classe=np.setdiff1d(posicoes_sem_classe,posicoes_classificadas)
        
    def score(self,x_teste,y_teste):
        print(accuracy_score(y_teste,self.modelo.predict(x_teste)))

In [25]:
modelo=classificador_semi_sup(7,KNeighborsClassifier)    

In [26]:
modelo.fit(x_treino,y_treino)

In [27]:
modelo.score(x_teste,y_teste)

0.9
